In [1]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import numpy as np
from numpy import var
from scipy import stats
from sklearn.preprocessing import normalize
from sklearn.metrics import cohen_kappa_score
from formfyxer import lit_explorer

/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
all_expert = pd.read_excel("RateMyPDF Individual Form Expert Benchmarks(1-176).xlsx")
form_column = "What form are you scoring? Please include the full URL, like: https://courtformsonline.org/forms/6e420f1b3575cfd8ef94b71977da9e38252e3395a78439709c760de4.pdf\n"
rating_column = "From 1-5, with 1 being the easiest and 5 being the hardest, how complex do you think this form is?\n "
name_column = "Full name"

In [3]:
df = all_expert[[form_column, rating_column, name_column]]
df["form_name"] = df[form_column].apply(lambda y: y.split("/")[-1].strip())
mean_and_stddev = (
    df.groupby(name_column)[rating_column]
    .agg(["mean", "std"])
    .rename(columns={"mean": "reviewer_mean", "std": "reviewer_stddev"})
    .reset_index()
)
df = pd.merge(df, mean_and_stddev, on=name_column)
df["z_score"] = (df[rating_column] - df["reviewer_mean"]) / df["reviewer_stddev"]
sorted_experts = (
    df.groupby("form_name")
    .mean()
    .sort_values(by="z_score", ascending=True)
    .reset_index()
)
# df.groupby(name_column).mean()

/tmp/ipykernel_3185248/3867465554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['form_name'] = df[form_column].apply(lambda y: y.split("/")[-1].strip())
/tmp/ipykernel_3185248/3867465554.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sorted_experts = df.groupby('form_name').mean().sort_values(by='z_score', ascending=True).reset_index()


In [4]:
import requests
import shutil


def calc_score(fname):
    # full_url = "https://courtformsonline.org/forms/" + fname
    # local_filename = "/tmp/" + fname
    # with requests.get(full_url, stream=True) as r:
    #  with open(local_filename, 'wb') as f:
    #    shutil.copyfileobj(r.raw, f)
    # The already downloaded and re-formated field PDFs, the ones that the stats are from in the paper.
    local_filename = "../../../tmp3/" + fname
    stats = lit_explorer.parse_form(local_filename)
    return lit_explorer.form_complexity(stats)


sorted_experts["our_complexity_score"] = sorted_experts["form_name"].apply(calc_score)

Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Se

In [5]:
sorted_experts["z_complexity_score"] = (
    sorted_experts["our_complexity_score"]
    - np.mean(sorted_experts["our_complexity_score"])
) / np.std(sorted_experts["our_complexity_score"])

In [6]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required
# to sort an array when only swapping of adjacent
# elements is allowed.
# include <bits/stdc++.h>


# This function merges two sorted arrays and returns inversion count in the arrays.*/
def merge(arr, temp, left, mid, right):
    inv_count = 0

    i = left  # i is index for left subarray*/
    j = mid  # i is index for right subarray*/
    k = left  # i is index for resultant merged subarray*/
    while (i <= mid - 1) and (j <= right):
        if arr[i] <= arr[j]:
            temp[k] = arr[i]
            k += 1
            i += 1
        else:
            temp[k] = arr[j]
            k += 1
            j += 1

            # this is tricky -- see above explanation/
            # diagram for merge()*/
            inv_count = inv_count + (mid - i)

    # Copy the remaining elements of left subarray
    # (if there are any) to temp*/
    while i <= mid - 1:
        temp[k] = arr[i]
        k += 1
        i += 1

    # Copy the remaining elements of right subarray
    # (if there are any) to temp*/
    while j <= right:
        temp[k] = arr[j]
        k += 1
        j += 1

    # Copy back the merged elements to original array*/
    for i in range(left, right + 1, 1):
        arr[i] = temp[i]

    return inv_count


# An auxiliary recursive function that sorts the input
# array and returns the number of inversions in the
# array. */
def _mergeSort(arr, temp, left, right):
    inv_count = 0
    if right > left:
        # Divide the array into two parts and call
        # _mergeSortAndCountInv()
        # for each of the parts */
        mid = int((right + left) / 2)

        # Inversion count will be sum of inversions in
        # left-part, right-part and number of inversions
        # in merging */
        inv_count = _mergeSort(arr, temp, left, mid)
        inv_count += _mergeSort(arr, temp, mid + 1, right)

        # Merge the two parts*/
        inv_count += merge(arr, temp, left, mid + 1, right)

    return inv_count


# This function sorts the input array and returns the
# number of inversions in the array */
def countSwaps(arr):
    n = len(arr)
    temp = [0 for i in range(n)]
    return _mergeSort(arr, temp, 0, n - 1)  # Time to get random sorts

In [9]:
# Time to get sorts
def get_avg_sort_count(df, final_order_col, sort_by, runs=1):
    swap_counts = []
    for i in range(runs):
        if sort_by == "random":
            array = df[final_order_col].sample(frac=1).values.copy()
        else:
            df = df.sample(frac=1).sort_values(by=sort_by, ascending=True)
            array = df[final_order_col].values.copy()
        swap_counts.append(countSwaps(array))
    return np.array(swap_counts).mean()


print(
    f"Random sort count: {get_avg_sort_count(sorted_experts, 'z_score', 'random', runs=5000)}"
)
print(
    f"Machine sort count: {get_avg_sort_count(sorted_experts, 'z_score', 'z_complexity_score', runs=5000)}"
)

Random sort count: 386.7084
Machine sort count: 299.0


In [12]:
# Time to get cohen's kappa
# put both ours and theirs back into bins.
bins = [-3, -2, -1, 0, 1, 2, 3]
expert_bins = np.digitize(sorted_experts["z_score"], bins=bins)
algo_bins = np.digitize(sorted_experts["z_complexity_score"], bins=bins)
print(
    f'scipy_cohen_kappa: {cohen_kappa_score(expert_bins, algo_bins, weights="quadratic")}'
)
# print(stats.ttest_rel())

scipy_cohen_kappa: 0.6206896551724137
